In [1]:
import sys

from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import Resize
import torch

sys.path.insert(0, '..')

from scripts.dataset_duplicate_image import DuplicateImageDataset
from scripts.model import SiameseDuplicateImageNetwork

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
model = SiameseDuplicateImageNetwork().to(device)

train_dataset = DuplicateImageDataset('../data/Training Data', transforms=[Resize((224, 224), antialias=True)])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8, persistent_workers=True)

test_dataset = DuplicateImageDataset('../data/Test Data', transforms=[Resize((224, 224), antialias=True)])
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=8, persistent_workers=True)

print(model)

SiameseDuplicateImageNetwork(
  (preprocess): ImageClassification(
      crop_size=[224]
      resize_size=[232]
      mean=[0.485, 0.456, 0.406]
      std=[0.229, 0.224, 0.225]
      interpolation=InterpolationMode.BILINEAR
  )
  (model): Sequential(
    (0): SiameseNetwork(
      (net1): Sequential(
        (0): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): Hardswish()
          )
          (1): InvertedResidual(
            (block): Sequential(
              (0): Conv2dNormActivation(
                (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
                (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
              )
              (1): Conv2dNo

In [4]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.RMSprop(
    [params for params in model.parameters() if params.requires_grad],
    lr=1e-3,
    alpha=0.9
)

In [5]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model.train_loop(train_dataloader, loss_fn, optimizer, device)
    model.test_loop(test_dataloader, loss_fn, device)
print("Done!")

Epoch 1
-------------------------------


C:\Users\ruiqinng\miniconda3\envs\dai\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


loss: 0.694239  [   64/32524]
loss: 0.339309  [ 6464/32524]
loss: 0.190897  [12864/32524]
loss: 0.184371  [19264/32524]
loss: 0.186786  [25664/32524]
loss: 0.040258  [32064/32524]
Test Error: 
 Accuracy: 71.7%, Avg loss: 0.830413 

Epoch 2
-------------------------------
loss: 0.238108  [   64/32524]
loss: 0.287254  [ 6464/32524]
loss: 0.047490  [12864/32524]
loss: 0.060416  [19264/32524]
loss: 0.095523  [25664/32524]
loss: 0.105319  [32064/32524]
Test Error: 
 Accuracy: 72.0%, Avg loss: 1.025935 

Epoch 3
-------------------------------
loss: 0.156774  [   64/32524]
loss: 0.028161  [ 6464/32524]
loss: 0.056886  [12864/32524]
loss: 0.108464  [19264/32524]
loss: 0.074857  [25664/32524]
loss: 0.001772  [32064/32524]
Test Error: 
 Accuracy: 72.6%, Avg loss: 1.479386 

Epoch 4
-------------------------------
loss: 0.008113  [   64/32524]
loss: 0.009574  [ 6464/32524]
loss: 0.019822  [12864/32524]
loss: 0.014388  [19264/32524]
loss: 0.003600  [25664/32524]
loss: 0.013464  [32064/32524]
Test